# Ankle Quasi Stiffness characterization from human gait data.

The purpose of this notebook is to show the main statitistical results of the ankle quasi-stiffness characterization from human gait data. Here, as a proof of concept, we tried to describe the ankle quasi-stiffness parameters for different anthropomorphical groups as well as many speed instances. More information is described in :

(Reference of my paper)

Herein, ...

## Loading required libraries

In [1]:
from DJSFunctions import extract_preprocess_data, ankle_DJS
from plot_dynamics import plot_ankle_DJS
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from scipy import stats
from utilities_QS import multi_idx, create_df, best_hyper, change_labels
import itertools as it
from itertools import permutations

# stats
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
import seaborn as sns
from scipy.stats.mstats import kruskal
import scikit_posthocs as sp
from statannotations.Annotator import Annotator  # Significant differences
sns.set_context('paper', font_scale=1.5)
sns.set_style("whitegrid")

## Some utilities for statistic calculations

In [2]:

# =============================================================================
# Class to plot the box plots
# =============================================================================

class obt_box_plot:
    def __init__(self, dep_vars, formal_labels, nrows=3, ncols=3, figsize=(12, 12),
                 wspace=0.3, left_space=0.1, labels_take=np.r_[:4, 5:11]):
        self.ncols = ncols
        self.nrows = nrows
        self.fig1, self.axes = plt.subplots(nrows, ncols, figsize=figsize)
        self.fig1.tight_layout()
        self.fig1.subplots_adjust(wspace=wspace, left=left_space)
        self.labels_to_take = labels_take
        self.axes = self.trim_axs(self.axes, len(self.labels_to_take))
        self.ax = None
        self.deps_mod_ = dep_vars[self.labels_to_take]
        self.labels_mod_ = np.array(formal_labels)[self.labels_to_take]
        self.hue_order = [] 

    def plot_boxplot(self, varx,  dataset, xlabel, hue='Speed', order=None,
                     hue_order=["VS", 'S', 'C', 'F', 'VF'], legend_pos=4, rot=0):
        self.hue = hue
        self.hue_order = hue_order
        for num,  self.ax in enumerate(np.ravel(self.axes)):
            if num < (self.nrows * self.ncols) - 1:
                sns.boxplot(x=varx, y=self.deps_mod_[num], hue=self.hue,
                            data=dataset, ax=self.ax, hue_order=hue_order, order=order)
                self.ax.set_ylabel(self.labels_mod_[num])
                if num != legend_pos:
                    self.ax.get_legend().remove()
                else:
                    self.ax.legend(loc='lower right')
                self.ax.set_xlabel(xlabel)
                if rot != 0:
                    plt.setp(self.ax.get_xticklabels(), rotation=rot)
            else:
                continue

        return 
    
    def annotate_(self):
        """
        Work in Progress. 
        To generate automatic stats annotations

        Returns
        -------
        None.

        """
        perm = permutations(self.hue_order)
        annotator = Annotator(self.ax, list(perm))
        annotator.set_pvalues()
        

    def trim_axs(self, axs, N):
        """
        Reduce *axs* to *N* Axes. All further Axes are removed from the figure.
        Copied from pyplot webpage
        """
        axs = axs.flat
        for ax in axs[N:]:
            ax.remove()
        return axs[:N]

    def save_fig(self, fig_name, sup_title=False):
        if sup_title:
            self.fig1.suptitle(
                'Variables with statistical differences in OvsT and speed', fontsize=18)
        self.fig1.savefig(fig_name)
        return self.fig1

# =============================================================================
# Function to do test student test
# =============================================================================


def ttest_(ds1, ds2, dep_vars):
    """


    Parameters
    ----------
    ds1 : Dataset 1
    ds2 : Dataset 2
    items : items in a dict format

    Returns
    -------
    None.

    """
    # Assumptions:
    #     1. Independent samples
    #     2. Large enough sample size or observations come from a normally-distributed
    #     population
    #     3. Variances are equal, if not apply weltch test

    # Does the samples come from a normally distributed population
    # Let's perform the Bartetts's test whose Null Hypothesis is that the
    # variances are equal. We will use a significance level of 5.0%, for lower values the null hypothesis is rejected
    # and the variances are not equal

    # Measuring and storing if the samples has the same variance
    var = {item: stats.bartlett(ds1[item],
                                ds2[item]).pvalue for item in dep_vars}
    # Performing the ttest, if not equal it will perform
    ttest_ = {item: stats.ttest_ind(ds1[item], ds2[item],
                                    equal_var=var[item] > 0.05).pvalue for item in dep_vars}
    return var, ttest_


# Testing normal distributions
# For values below 5% the hipothesis is rejected and is non-normal distribution
def shapiro_test(ds, dep_vars, name='No name', df=True):

    if df == True:
        shapiro_ = {item: stats.shapiro(
            ds[item]).pvalue > 0.05 for item in dep_vars}
        shapiro_df = pd.Series(shapiro_, name=name)
        return shapiro_df
    else:
        shapiro_ = {item: stats.shapiro(ds[item]).pvalue for item in dep_vars}
        return shapiro_

# =============================================================================
#    Kruskal Wallis test on ranks
# =============================================================================


def kruskal_groups(ds1, ds2, dep_vars, name):
    kruskal_deps = pd.Series({item: kruskal(ds1[item].values, 
                                              ds2[item].values).pvalue < 0.05 for item in dep_vars})
    kruskal_deps = kruskal_deps.replace(True,1)
    kruskal_deps = kruskal_deps.replace(False,0)
    kruskal_deps.name = name
    return kruskal_deps



In [ ]:
concat_QS = pd.read_csv('PostProcessedDatasets.csv', index_col=[0])